In [24]:
import pandas as pd

# getting the A.Revel kaggle dataset. focus on AN and ELO for white and black (no matter who won)
raw = pd.read_csv("/Volumes/vrona_SSD/lichess_data/arevel_db.csv", usecols=['AN', 'WhiteElo', 'BlackElo'])

In [33]:
# keeping only ELO above 2000
wb_2000 = raw[(raw['WhiteElo'] > 2000) & (raw['BlackElo'] > 2000)]
wb_2000.shape

(626180, 3)

In [34]:
# remove uncessary character
wb_2000 = wb_2000[~wb_2000['AN'].str.contains('{')]
wb_2000.shape


(548141, 3)

In [35]:
# keeping only game with more than 300 character (more than 40 moves)
wb_2000 = wb_2000[wb_2000['AN'].str.len() > 300]
wb_2000.shape


(445539, 3)

In [16]:
# checking how it looks like
wb_2000.head()

,WhiteElo,BlackElo,AN
7,2155,2356,1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5. Be2...
8,2010,2111,1. d4 Nf6 2. Bf4 e6 3. e3 d5 4. Nf3 h6 5. Bd3 ...
26,2149,2047,1. e4 d6 2. d4 Nf6 3. Bd3 g6 4. h4 c5 5. h5 Nx...
39,2215,2072,1. e4 e6 2. d4 d5 3. Nc3 Bb4 4. e5 c5 5. a3 cx...
44,2053,2056,1. d4 d5 2. Nc3 c6 3. Nf3 Nf6 4. Bg5 e6 5. e3 ...


In [17]:
# saving as csv
wb_2000_wthidx = wb_2000.to_csv("lichess_data/wb_2000_noidx.csv", index=False)

In [18]:
# check the saving
wb_2000_noidx = pd.read_csv("lichess_data/wb_2000_noidx.csv")
wb_2000_noidx.head()

,WhiteElo,BlackElo,AN
0,2155,2356,1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5. Be2...
1,2010,2111,1. d4 Nf6 2. Bf4 e6 3. e3 d5 4. Nf3 h6 5. Bd3 ...
2,2149,2047,1. e4 d6 2. d4 Nf6 3. Bd3 g6 4. h4 c5 5. h5 Nx...
3,2215,2072,1. e4 e6 2. d4 d5 3. Nc3 Bb4 4. e5 c5 5. a3 cx...
4,2053,2056,1. d4 d5 2. Nc3 c6 3. Nf3 Nf6 4. Bg5 e6 5. e3 ...


In [26]:
# dropping WhiteElo	BlackElo
wb_2000_noidx = wb_2000_noidx.drop(["WhiteElo", "BlackElo"], axis=1)
wb_2000_noidx.head()

,AN
0,1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5. Be2...
1,1. d4 Nf6 2. Bf4 e6 3. e3 d5 4. Nf3 h6 5. Bd3 ...
2,1. e4 d6 2. d4 Nf6 3. Bd3 g6 4. h4 c5 5. h5 Nx...
3,1. e4 e6 2. d4 d5 3. Nc3 Bb4 4. e5 c5 5. a3 cx...
4,1. d4 d5 2. Nc3 c6 3. Nf3 Nf6 4. Bg5 e6 5. e3 ...


In [5]:
# testing & splitting the dataset into 3 sets (could be done with sklearn.model_selection .train_test_split)
import numpy as np
training_set, valid_set, test_set = np.split(wb_2000_noidx.sample(frac=1, random_state=42), [int(.6*len(wb_2000_noidx)), int(.8*len(wb_2000_noidx))])

                                                       AN
188632  1. d4 c5 2. d5 d6 3. c4 Nf6 4. Nc3 Bg4 5. e4 B...
173375  1. e4 c5 2. b3 Nc6 3. Bb2 e5 4. Bb5 Nge7 5. Ne...
163782  1. e4 a6 2. d4 b5 3. a4 b4 4. c3 a5 5. cxb4 ax...
364799  1. Nf3 g6 2. d4 Bg7 3. c3 c5 4. Bf4 cxd4 5. cx...
436504  1. g3 d5 2. Bg2 e6 3. b3 c5 4. Bb2 Nc6 5. Nf3 ...


In [6]:
# if needed saving each datasets as csv
training_set.to_csv("lichess_data/chess_training_set.csv", index=False)
valid_set.to_csv("lichess_data/chess_valid_set.csv", index=False)
test_set.to_csv("lichess_data/chess_test_set.csv", index=False)

In [15]:
print(trainset.shape,validset.shape,testset.shape)

(327868, 1) (109289, 1) (109290, 1)


In [21]:
# testing SQL
import sqlite3
from pathlib import Path

# create empty .db file
Path('chess_wb2000_train.db').touch()
Path('chess_wb2000_valid.db').touch()
Path('chess_wb2000_test.db').touch()

#data = pd.read_csv("lichess_data/wb_2000.csv")

In [45]:
# connect to sqlite
conn = sqlite3.connect('chess_wb2000_db.db')
cur = conn.cursor()

In [47]:
wb_2000_noidx.columns

Index(['AN'], dtype='object')

In [46]:
# create table
cur.execute("CREATE TABLE games (id INTEGER PRIMARY KEY, AN text)")

In [48]:
# tricky part keep the index from DF with index=True but rename the column of index by "id"
wb_2000_noidx.to_sql('games', conn, if_exists='append', index=True, index_label='id')

546447

In [17]:
# industrialization of above
def create_db(dataframe, name):
    conn = sqlite3.connect(name)
    cur = conn.cursor()
    cur.execute("CREATE TABLE games (id INTEGER PRIMARY KEY, AN text)")
    dataframe.to_sql('games', conn, if_exists='append', index=True, index_label='id')

In [22]:
listofdataframe = [trainset,validset,testset]
listofname = ['chess_wb2000_train.db','chess_wb2000_valid.db','chess_wb2000_test.db']
for df, x in zip(listofdataframe, listofname):
    create_db(df, x)